# 利用可再生能源和能源储存来减少电力消耗

## 简介

随着太阳能和风能发电技术的采用,使用可再生能源已经越来越受欢迎。然而,太阳并不总是照耀,风也不总是吹动。当客户需要能源时,可能与风能和太阳能潜力高的时段不重合,这是可再生能源面临的一个关键挑战。电池可以成为解决这个问题的方案,通过放电使用储存的能量,使其可以在一天中的任何时间使用。

在这个例子中,我们将利用太阳能和电池储能来满足运营一座大学建筑及其相关课程的能源需求。在接下来的部分中,我们将描述问题陈述,并演示如何使用数据科学和数学优化来最优地解决给定问题。

本示例使用的信息来自[IEEE的预测+优化技术挑战赛](https://ieee-dataport.org/competitions/ieee-cis-technical-challenge-predictoptimize-renewable-energy-scheduling)。

## 目标和前提条件

在这个例子中,我们考虑澳大利亚墨尔本校区的一座大学建筑,该建筑在六天内(周一至周六)安排了一系列课程。

建筑物和其中举办的每门课程都有需要满足的特定能源需求。建筑物的能源需求假定是已知的,每门课程根据所需资源(如班级规模或实验室)有不同的需求。该建筑安装了太阳能板,可以直接提供能量,同时也可以直接从电网购买电力来满足需求。此外,该建筑配备了两个电池,可以储存太阳能板提供的能量,并在需要时提供这些能量。我们还允许电池从电网充电。

本示例将把时间序列数据预测与数学优化相结合,因为我们将把太阳能发电预测作为优化问题的关键输入。 

## 问题陈述

给定下周的课程安排和历史太阳能潜力数据,目标是确定最佳的电池充放电计划,以满足举办课程的建筑物的能源需求。总能源需求由建筑物本身的需求加上每门课程所需的能源量组成。该示例将考虑两个目标:
- 最小化下周从电网购买的总电量
- 由于电价波动,最小化从电网购买的电力总成本

同时,电池具有有限的容量以及在一段时间内可以充电或放电的限制。这些约束使问题变得难以解决。

为简单起见,每天考虑的时间将限制在早上5点至晚上8点之间。如果您可以访问完整的Gurobi许可证(例如通过我们的[学术计划](https://www.gurobi.com/academia/academic-program-and-licenses/)),请随时扩展此范围以解决更大规模的问题。

这里是一周课程安排的视图:
![sched](ClassSchedule.png)

## 解决方案方法

问题的解决方案方法包含两个组成部分: 1) **太阳能可用性预测组件** 和 2) **优化组件**,用于确定电池计划以及从电网购买的电量。

### 太阳能发电预测

预测组件在本文件夹中的[energy_storage_ML](energy_storage_ML.ipynb)笔记本中完成。我们将使用该模型的给定预测,以及建筑物和每门课程的需求,使用gurobipy制定和解决混合整数规划(MIP)问题,为上述每个目标找到最优解。

我们首先通过安装和加载需要的包来开始我们的解决方案方法。 

In [ ]:
%pip install gurobipy
import pandas as pd
import matplotlib.pyplot as plt
import gurobipy as gp
from gurobipy import GRB

batteries = ["Battery0", "Battery1"]
path = 'https://raw.githubusercontent.com/Gurobi/modeling-examples/master/optimization101/Modeling_Session_2/'
solar_values_read = pd.read_csv(path+'pred_solar_values.csv')
# solar_values_read = pd.read_csv('pred_solar_values.csv')
time_periods = range(len(solar_values_read))

### 输入参数

每个电池都有一个固定的容量,表示它可以储存多少能量,以及每30分钟内可以储存或放电的最大能量。

$c_{b}$: 电池 $b \in B$ 的容量 $\quad\quad \texttt{capacity[b]}$

$p_{b}$: 电池 $b\in B$ 充电过程中的能量损耗(百分比) $\quad\quad \texttt{p}\_\texttt{loss[b]}$

$q_{b}$: 电池 $b \in B$ 的初始能量 $\quad\quad \texttt{initial[b]}$

$solar_{t}$: 时间段 $t \in T$ 的太阳能板发电量 $\quad\quad \texttt{solar}\_\texttt{values[b]}$

In [ ]:
capacity = {"Battery0": 60, "Battery1": 80} # 单位:千瓦
p_loss = {"Battery0": 0.95, "Battery1": 0.9} # 占比
initial = {"Battery0": 0, "Battery1": 0} # 单位:千瓦

solar_values = round(solar_values_read.yhat,3)
solar_values.reset_index(drop = True, inplace = True)

特定时间段的能源需求来自两个来源 -- 建筑物和课程 -- 我们只关注总需求。这些数据将从外部文件读取,并且已按时间段正确排序。

$d_{t}$: 时间段 $t\in T$ 的建筑物和课程总能源需求 $\quad\quad \texttt{total}\_\texttt{deamnd[t]}$

In [3]:
schedule = pd.read_csv(path+'schedule_demand.csv')
# schedule = pd.read_csv('schedule_demand.csv')
avg_building = pd.read_csv(path+'building_demand.csv')
# avg_building = pd.read_csv('building_demand.csv')
total_demand = schedule.sched_demand + avg_building.build_demand
print(f"Total Solar Generation: {solar_values.sum()} \nTotal Demand: {total_demand.sum()}")

### 决策变量

本问题的决策包括:
- 每个电池在给定时间段是充电还是放电
- 每个电池充电或放电多少
- 每个电池在给定时间段的当前状态
- 以及从电网购买多少能量

令 $f^{in}_{b,t}$ 为每个电池 $b$ 在时间段 $t$ 的`充电`量, $\forall b\in B, t\in T$。 $\quad\quad \texttt{flow}\_\texttt{in[b,t]}$

令 $f^{out}_{b,t}$ 为每个电池的`放电`量, 类似定义。 $\quad\quad \texttt{flow}\_\texttt{out[b,t]}$

设定每个电池在单个时间段内可充电或放电的最大量为20千瓦。

In [ ]:
m = gp.Model() #这定义了我们将在完成建模时添加内容的模型

flow_in = m.addVars(batteries, time_periods, name="flow_in") 
flow_out = m.addVars(batteries, time_periods, name="flow_out")

接下来是每个时间段从电网购买的能量。我们假设这是一个非负数,并且我们不能"卖回"给电网(尽管这也是一个有趣的问题!)

$grid_{t}$: 这个变量表示在时间段 $t$ 从电网购买的能量量, $\forall t \in T$

In [5]:
grid = m.addVars(time_periods, name="grid")

接下来两组决策变量会有略微不同的感觉。我们需要追踪每个电池在每个时间段内的能量量(即其状态),并且需要追踪使用了多少太阳能(通过充电或直接供应能量)。

$s_{b,t}$ 是电池 $b$ 在时间段 $t$ 结束时的当前能量, $\forall b\in B, t\in T$。 $\quad\texttt{state[b,t]}$

$gen_{t}$ 是时间段 $t$ 中使用的可用太阳能量, $\forall t \in T$。 $\quad\texttt{gen[t]}$


In [6]:
state = m.addVars(batteries, time_periods, name="state") 
gen = m.addVars(time_periods, name="gen")

最后要添加的决策变量将处理这类问题中可能出现的一个棘手情况 -- 电池不能同时充电和放电。因此,我们需要确保每个电池在每个时间段都只做以下三件事之一: 1. **充电**, 2. **放电**, 或 3. **两者都不做**。

为了对此建模,我们为每个时间段引入一个二进制变量 $z_{b,t}, \forall b \in B, t \in T$,我们将其称为 $\texttt{zwitch}\_\texttt{[b,t]}$。为什么叫*zwitch*?这是我从早期优化建模时就开始使用的,因为在数学优化中常用于这类二进制变量的单字母是 $z$。这个变量的作用像开关 -- 因此叫*zwitch*。

In [7]:
zwitch = m.addVars(batteries, time_periods, ??? , name="zwitch")

### 约束条件
在每个时间段,我们需要确保满足能源需求。可用的能源可以来自三个来源: 1) 电网, 2) 太阳能板, 或 3) 电池放电。此外,我们可能决定给电池充电,这将有助于在以后的时间段满足需求。

$$
\begin{align*} 
\sum_b(f^{out}_{b,t}-p_bf^{in}_{b,t}) + gen_t + grid_t = d_t \quad \forall t \in T
\end{align*}
$$

In [8]:
m.addConstrs((gp.quicksum( ???
                  for t in time_periods), name="power_balance");

每个电池在给定时间段的状态会影响下一个时间段的状态,以及充电或放电的决策。因此,一般来说,电池在时间 $t$ 的状态是前一时间段的状态以及任何充电或放电。

为此,我们根据每个电池的初始能量以及第一个时间段的充电/放电决策设置初始状态。

\begin{equation}
???
\end{equation}

对于之后的每个时间段(即 $t\ge1$),电池的状态由以下公式确定:

\begin{equation}
???
\end{equation}

相应的约束条件可以写成如下。

In [9]:
m.addConstrs((state[b,0] == initial[b] + p_loss[b]*flow_in[b,0] - flow_out[b,0] for b in batteries), name="initial_state")
m.addConstrs((state[b,t] == state[b,t-1] + p_loss[b]*flow_in[b,t] - flow_out[b,t] for b in batteries for t in time_periods if t >= 1), name="subsequent_states");

太阳能的可用性可以用于三个地方:
1. Battery0
2. Battery1
3. 直接满足需求

因此,我们需要限制电池的充电量以及直接用于需求的量。

$$
\begin{equation}
???
\end{equation}
$$

In [10]:
m.addConstrs((   ???   ), name = "solar_avail");

### 二进制变量建模逻辑
使用二进制变量来建模像我们这里这样的替代方案可能是数学优化建模中最困难的部分。这归结为能够使用零、一和不等式来表示复杂的系统。要正确地理解逻辑需要时间,并且在这一点上有很多迭代以确保正确完成。简而言之,如果你发现这部分困难,请不要担心。

让我们制定约束条件,然后深入探讨它们的工作原理。

$$
\begin{align*} 
f^{in}_{b,t} &\leq 20*z_{b,t} &\forall b \in B, t \in T \\
f^{out}_{b,t} &\leq 20*(1-z_{b,t}) &\forall b \in B, t \in T
\end{align*}
$$

为了说明其工作原理,让我们考虑一个例子 -- 假设在一个时间段内,某个电池的 $f^{in}_{b,t} = 10$。为了使第一个不等式成立,有必要 $z_{b,t} = 1$, 否则它是 $0$ 并且 $20*0 = 0$, 这不可能发生如果 $f^{in}_{b,t} = 10$。然后在同一时间段,如果 $z_{b,t} = 1$, 那么 $1-z_{b,t} = 0$。这迫使第二个不等式的右侧为 $0$, 这迫使 $f^{out}_{b,t} = 0$, 使得在此期间放电变得不可能。类似的逻辑适用于电池在一个时间段内放电的情况(即假设 $f^{out}_{b,t} = 10$)。

由于我们的 `zwitch` 是一个二进制变量,这意味着每个时间段的两种情况之一将始终发生。重要的问题是 -- 这是一个问题吗?答案是否定的,因为我们需要担心的是我们的不等式*强制*决策变量是什么。考虑电池在一个时间段内既不充电也不放电的情况(即流入和流出为零)。这种情况不会违反上述任何一个不等式,并且无论 $z$ 取何值 -- 它可以是 $0$ 或 $1$, 不等式都成立。

这确实提出了一个重要的问题。如果在解决此问题后有人问你"电池充电和放电的时间段是多少?"很容易认为 $z_{b,t} = 1$ 意味着电池正在充电,因此只需将这些加起来即可得到充电时间段的数量。然后计算 $z_{b,t} = 0$ 的次数以获得放电计数。这将是*不正确的*,因为我们看到 $f^{in}_{b,t} = f^{in}_{b,t} = 0$ 时的情况。

20 是从哪里来的?这是电池在一个时间段内可以充电或放电的上限。

In [11]:
m.addConstrs((flow_in[b,t] <= 20*zwitch[b,t] for b in batteries for t in time_periods), name = "to_charge")
m.addConstrs((flow_out[b,t] <= 20*(1-zwitch[b,t]) for b in batteries for t in time_periods), name = "or_not_to_charge");

最后,我们需要设置每个电池可以储存的最大量,这意味着电池的*状态*必须有上限。这些上限可以在使用 `addVars()` 添加变量时设置。在那里这样做需要一个与决策变量维度相同的列表。下面是另一种使用 `for` 循环快速设置上限的方法。有关添加变量时可以设置的所有参数的更多信息,请参见[文档]('https://www.gurobi.com/documentation/10.0/refman/py_model_addvars.html')。通常,在添加变量时设置上限更有效。

In [12]:
for b, t in state:
    state[b,t].UB = capacity[b]

$$
\begin{align*} 
s_{b,t} \le c_b, \quad \forall b \in B
\end{align*}
$$

### 目标函数

目标是最小化所有时间段从电网购买的总能量。

\begin{equation}
{\rm minimize} \quad \sum_{t} g_{t}
\end{equation}

In [13]:
m.setObjective(gp.quicksum(grid[t] for t in time_periods), GRB.MINIMIZE)

### 求解优化模型和输出分析
现在模型已经设置好,我们可以优化以最小化购买的电力。

In [ ]:
m.optimize()
print(f"Total energy purchased from the grid: {round(m.objVal,3)} kWh")

现在让我们看看一些决策变量的值。使用 *getAttr* 函数和 *'X'* 参数来获取 `state` 决策变量的值。

In [ ]:
sol_in = pd.Series(m.getAttr('X',flow_in))
sol_out = pd.Series(m.getAttr('X',flow_out))
sol_level = pd.Series(m.getAttr('X',state))

print(f"Periods Battery0 charges: {sum(sol_in['Battery0'] > 0)}")
print(f"Periods Battery1 charges: {sum(sol_in['Battery1'] > 0)}")
print(f"Periods Battery0 discharges: {sum(sol_out['Battery0'] > 0)}")
print(f"Periods Battery1 discharges: {sum(sol_out['Battery1'] > 0)}")

每个电池状态随时间的变化图:

In [ ]:
plt.figure(figsize=(12,5))
s0, = plt.plot(sol_level['Battery0'], c = 'orange')
s1, = plt.plot(sol_level['Battery1'], c = 'blue')
plt.ylabel('Battery State (kWhr)')
plt.xlabel('Time Period')
plt.legend([s0,s1],["Battery0", "Battery1"])
plt.axhline(y=capacity['Battery0'], c='orange', linestyle='--', alpha = 0.5)
plt.axhline(y=capacity['Battery1'], c='blue', linestyle='--', alpha = 0.5)
print(f"Periods at Battery0 at Full Capacity: {sum(sol_level['Battery0']==capacity['Battery0'])}")
print(f"Periods at Battery1 at Full Capacity: {sum(sol_level['Battery1']==capacity['Battery1'])}");

使用默认的容量值,较小容量的电池(Battery0)仅在两个时间段达到其容量,而Battery1更频繁地达到其上限。更改这些值会如何改变优化结果?

我们最后一组决策变量是从电网购买的电力量,其总量在目标函数中被最小化,并绘制其随时间的变化图。

In [ ]:
sol_grid = pd.Series(m.getAttr('X',grid))
plt.figure(figsize=(12,5))
plt.plot(sol_grid)
plt.ylabel('Power Purchased (kWhr)')
plt.xlabel('Time Period');
print(f"Proportion of time periods where electrity is purchaced from the grid: {round(sum(sol_grid > 0)/len(sol_grid),3)}")

上面计算了从电网购买电力的时间段数量。这与最小化购买的总量相同吗?如果不是(提示,不是!),你会如何建模? (另一个提示,这将需要额外的二进制变量)
### 更改目标函数
如示例开头所述,我们将考虑两个目标函数,第二个目标是最小化购买电力的成本。为此,我们需要每个时间段的预期价格。

In [18]:
avg_price = pd.read_csv(path+'expected_price.csv')
# avg_price = pd.read_csv('expected_price.csv')
plt.figure(figsize=(12,5))
plt.plot(avg_price.index, avg_price.price, '-o');

要更改目标函数,只需重新设置它。模型的其余部分保持不变,因此不需要进一步调整。

In [19]:
m.setObjective(gp.quicksum(avg_price.price[time]*grid[time] for time in time_periods), GRB.MINIMIZE)

In [ ]:
m.optimize() # 再次运行优化
print(f"Energy cost for the week: ${round(m.objVal,2)}")

我们可以像之前一样提取决策变量,看看给定新目标的解决方案如何变化。

In [21]:
sol_level_cost = pd.Series(m.getAttr('X',state))
plt.figure(figsize=(12,5))
plt.plot(sol_level_cost['Battery0'], c ='orange')
plt.plot(sol_level_cost['Battery1'], c ='blue')
plt.legend([s0,s1],["Battery0", "Battery1"])
plt.axhline(y=capacity['Battery0'], c='orange', linestyle='--', alpha = 0.5)
plt.axhline(y=capacity['Battery1'], c='blue', linestyle='--', alpha = 0.5)
print(f"Periods at Battery0 at Full Capacity: {sum(sol_level_cost['Battery0']==capacity['Battery0'])}")
print(f"Periods at Battery1 at Full Capacity: {sum(sol_level_cost['Battery1']==capacity['Battery1'])}");

我们看到每个电池花费更多时间处于满或接近满的状态,特别是Battery0。这表明添加成本将使电池保留电力,以便在成本最高的时间段放电。

以下是一些深入探讨此问题的后续问题:
- 更改目标后从电网购买了多少电力?
- 使用第二个目标的价格,第一个最小化购买电力的解决方案的成本是多少?
- 在第二个目标中增加Battery0的容量会如何改变解决方案?

<h2>结论</h2>

本示例演示了如何建模利用时间序列预测太阳能可用性的能源系统。建模此类系统的挑战之一是一个时间段的决策如何影响下一个时间段。我们还看到了定义决策变量边界的两种方法,并且看到更新目标函数非常简单,可能对解决方案产生重大影响。Gurobi的广泛[示例笔记本](https://www.gurobi.com/resource/modeling-examples-using-the-gurobi-python-api-in-jupyter-notebook/)中有更多[与能源相关](https://www.gurobi.com/resource/electrical-power-generation-jupyter-notebook-i-and-ii/)的示例。

版权所有 © 2022 Gurobi Optimization, LLC